<a href="https://colab.research.google.com/github/Lima-Ricardo/Analise-Pandas-Pyspark-GCP/blob/main/Projeto_Individual_Pandas_E_Pyspark_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Item postado em 09:36 Editado às 09:45
Atribuído
Nivel Infra
O Dataset deve ser salvo em ambiente cloud(Cloud Storage)
O arquivo original e tratado deve ser salvo em MongoDB Atlas em coleções diferentes
Os DataFrames devem ser obrigatoriamente salvos em uma bucket do CloudStorage

Nivel Pandas
O arquivo está em outra linguagem e deve ter seus dados traduzidos para Português-BR
Realizar a extração corretamente para um dataframe
Verificar a existência de dados inconsistentes e realizar a limpeza para NaN ou NA
Realizar o drop(se necessário) de colunas do dataframe realizando o comentário do porque da exclusão 
Todos os passos devem ser comentados
Nivel PySpark (Funções básicas vistas em aula)
Deverá ser montada a estrutura do DataFrame utilizando o StructType.
Verificar a existência de dados inconsistentes, nulos e realizar a limpeza.
Verificar a necessidade de drop em colunas ou linhas. Caso seja necessário, fazer comentário do porque.
Realizar a mudança de nome de pelo menos 2 colunas
Deverá criar pelo menos duas novas colunas contendo alguma informação relevante sobre as outras colunas já existentes (Funções de Agrupamento, Agregação ou Joins). (Use a sua capacidade analítica)
Deverá utilizar filtros, ordenação e agrupamento, trazendo dados relevantes para o negócio em questão. (Use a sua capacidade analítica)
Utilizar pelo menos duas Window Functions
Nivel SparkSQL
Utilizar no minimo 5 consultas diferentes utilizando o SparkSQL, comentando o porquê de ter escolhido essas funções e explicando o que cada consulta faz.
Nível DataStudio
Construir um dashboard (maximo 1 pagina) para apresentação dos insights

Ferramentas


Colab ou Ides | Google Cloud | Data Studio***

***Intalando as libs necessárias para o projeto***

In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 17.6 MB/s 
     |████████████████████████████████| 140 kB 19.5 MB/s 
     |████████████████████████████████| 144 kB 16.8 MB/s 
     |████████████████████████████████| 271 kB 54.5 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 


In [ ]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 39.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=52ea26654734629bfa8fac0c0d8fede5f2b8984a3f4c4e026996ca66916b99d1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!pip uninstall googletrans

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.7/dist-packages/googletrans-4.0.0rc1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/googletrans/*
Proceed (y/n)? y
  Successfully uninstalled googletrans-4.0.0rc1


In [ ]:
!pip3 install googletrans==3.1.0a

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached googletrans-4.0.0rc1-py3-none-any.whl


In [ ]:
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 27.1 MB/s 
     |████████████████████████████████| 11.1 MB 38.7 MB/s 


In [ ]:
!pip install translate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from google.cloud import storage
import googletrans 
from googletrans import Translator
from pymongo import MongoClient
from datetime import date
import os
import gcsfs
# from translate import Translator
# import translate

In [ ]:
#Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#CONFIGURAÇÃO DA CHAVE DE SEGURANÇA
serviceAccount = '/content/drive/MyDrive/Acesso GCP/hopeful-list-349119-f034072f7a24.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#CÓDIGO QUE ACESSA MEU BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS VIA PANDAS
client = storage.Client()

bucket = client.get_bucket('projetoindividual')

bucket.blob('marketing_campaign.csv')

path = 'gs://projetoindividual/Original/marketing_campaign.csv'


***Carregadno o CSV direto do Claudio***

In [ ]:
df = pd.read_csv(r'gs://projetoindividual/Original/marketing_campaign.csv', sep=';', encoding='utf-8')

***Fazendo uma pré análise dos dados obtidos***

In [ ]:
#Função utilizada para filtrar e calcular a quantidade de resgistros em branco, e a proporção dessa perda antes do ETL.
def dados_perdidos(df):
    perdas = df.isna().sum()
    perdas = perdas[perdas >0 ]
    perdas_proporcionais = perdas/df.shape[0]
    perdas_totais = perdas_proporcionais > 0.05

    return pd.DataFrame({"Perdas" : perdas, "Proporção de perdas" : perdas_proporcionais, "Perdas maiores que 5%" : perdas_totais})


dados_perdidos(df)

,Perdas,Proporção de perdas,Perdas maiores que 5%
Income,24,0.010714,False


In [ ]:
#Fazendo um backup do dataframe
df1_pt = df.copy()

***O csv está com a liguagem em inglês ('en'), também foram encontradas as seguintes inconsistências:
"COLUNA EDUCAÇÃO = AQUELE CICLO", 
"COLUNA ESTADO CIVIL = JUNTOS",
"COLUNA RENDA POSSUI 24 CAMPOS EM BRANCO" 
"COLUNA = DT_CUSTORMER, ESTÁ EM FORMATO OBJECT".***

PREPARANDO CORREÇÕES E APLICANDO A TRADUÇÃO NO DATAFRAME

In [ ]:
#Conferindo as siglas correspondentes para a tradução
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [ ]:
#Atribuindo o translator em uma variável
tradutor = Translator()


In [ ]:
#Traduzindo as minhas colunas para o formato PT 
try:
  df1_pt.rename(columns=lambda x: tradutor.translate(x, src='en', dest='pt').text, inplace=True)
except Exception as e:
  print(str(e))

In [ ]:
#Traduzindo todas as linhas para o formato PT 

#Criando um dicionário vazio
traducoes = {}

#Estrutura de repetição que vai percorrer verificar e todas colunas no dataframe
for colunas in df1_pt.columns:
    
    #Procurando elementos unicos no dataframe
    elementos_unicos = df1_pt[colunas].unique()
    
    #Estrutura de repetição que vai percorrer cada um dos elementos, traduzi-los e adicionar essa tradução dentro do meu dicionário
    for elemento in elementos_unicos:
        traducoes[elemento] = tradutor.translate(elemento, dest='pt').text

TypeError: ignored

In [ ]:
df1_pt.replace(traducoes, inplace = True)

In [ ]:
#Corrigindo manualmente a falha de tradução do google translate
colunas_traduzidas = (['ID', 'Ano_Nascimento', 'Escolaridade', 'Estado_civil', 'Renda', 'Qtde_Crianças_no_lar', 'Qtde_Adolescentes_no_lar',
             'Data_do_cadastro', 'Dias_da_ultima_compra', 'Gasto_com_vinho', 'Gasto_com_frutas', 'Gasto_com_carnes', 'Gasto_frutos_do_mar', 'Gasto_com_doces', 
             'Produtos_ouro', 'Compras_com_desconto', 'Compras_online', 'Compras_no_catalogo', 'Compras_na_loja', 'Visitas_no_website', 'campanhas_aceitas_01',
             'Campanhas_aceitas02', 'Campanhas_aceitas_03', 'Campanhas_aceitas_04', 'Campanhas_aceitas_05', 'Reclamações', 'Custo_do_Contato','Receita', 'Oferta_aceita']
    
)
df1_pt.columns = colunas_traduzidas
df1_pt

In [ ]:
#Aplicando replace nos campos que não foram traduzidos corretamente, e corrigindo campos com erro de escrita
df1_pt.replace(to_replace = 'Juntos', value = 'União Estável', inplace=True)
df1_pt.replace(to_replace = 'aquele ciclo', value = 'N/E', inplace=True)


In [ ]:
#Fazendo a correção da coluna Data_do_cadastro que estava no tipo de texto para o tipo DATE-TIME
df1_pt['Data_do_cadastro'] = pd.to_datetime(df1_pt['Data_do_cadastro'], errors='coerce')


In [ ]:
#Aplicando tratamento para os campos nulos
# df1_pt.replace({"":np.nan})
df1_pt.fillna(value = 0, inplace=True)

In [ ]:
#Chamando a função novamente para fazer um novo cálculo apos o ETL.
dados_perdidos(df1_pt)

***USANDO O SPARK PARA TRATAR DADOS QUE NÃO FORAM POSSÍVEIS COM O PANDAS***

In [ ]:
#CHAMAR AS BIBLIOTECAS/MÓDULOS NECESSÁRIAS Pyspark
from pyspark import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType

In [ ]:
#CRIAR A SPARK SESSION E LER O ARQUIVO VIA PYSPARK
spark = (
    SparkSession.builder
                .master('local')
                .appName('spark-gcs')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [ ]:
#Puxando o dataframe tratado em pandas e transformando em um dataframe Pyspark
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df_spark = spark.createDataFrame(df1_pt)

***Fazendo uma análise pós tratamento com pandas.***

In [ ]:
df_spark.printSchema()

***Removendo colunas que apresentam inconsistências e que podem gerar erro na análise final.***
***Colunas: ID, Custo do contato, Receita***

In [ ]:
#Dropando a colunas ID, Custo do contato, Receita 
df_spark = df_spark.drop(F.col('ID'))
df_spark = df_spark.drop(F.col('Custo_do_Contato'))
df_spark = df_spark.drop(F.col('Receita'))

In [ ]:
# Convertendo a coluna para o tipo DATE
df_spark.select(F.col('Data_do_Cadastro'), F.to_date(F.col('Data_do_Cadastro'), 'yyyy-MM-dd')).printSchema()
df_data = df_spark.select(F.col('Data_do_Cadastro'), F.to_date(F.col('Data_do_Cadastro'), 'dd-MM-yy').alias('Data_do_cadastro_BR'))

In [ ]:
df_spark.select(F.col('Data_do_Cadastro'), F.date_format(F.col('Data_do_Cadastro'), 'dd-MM-yyyy').alias('Data_do_cadastro_BR')).printSchema()

In [ ]:
df_spark.withColumn('Data_do_cadastro_BR', F.current_date()).show()

In [ ]:
#Organizando o schema pelo ano de nascimento
df_spark.orderBy(F.col('Ano_Nascimento').desc()).show(100, truncate=False)

In [ ]:
#Removendo o registro 'Pedalar" da coluna Escolaridade
df_spark = ( df_spark.withColumn('Escolaridade', F.regexp_replace(F.col('Escolaridade'), 'pedalar', ''))
                      .withColumn('Renda', F.regexp_replace(F.col('Renda'), 'dentro', ''))
)

In [ ]:
#SUBSTITUIÇÃO DE VALORES NULOS
df_spark.fillna("NA", subset=['Escolaridade', 'Renda']).show()

In [ ]:
#Removendo duplicatas nos resgistros
df_spark.dropDuplicates().count()

In [ ]:
#Verificando a quantidade dos dados duplicados
total = (df_spark.count()) - (df_spark.dropDuplicates().count())
print(total)

In [ ]:
# #Alterando o nome das colunas
# df_spark = df_spark.withColumnRenamed("Qtde_Crianças_no_lar", "Qtde_Crianças_em_casa")
# df_spark = df_spark.withColumnRenamed("Qtde_Adolescentes_no_lar", "Qtde_Adolescentes_em_casa")

In [ ]:
#Criando uma nova coluna com o total de compras feitas
df_spark = df_spark.withColumn('total_compras', F.col('Compras_online') + F.col('Compras_no_catalogo') + F.col('Compras_na_loja') + F.col('Compras_com_desconto'))

In [ ]:
#Criando uma nova coluna com total de gastos
df_spark = df_spark.withColumn('total_gastos', F.col('Gasto_com_vinho') + F.col('Gasto_com_frutas') + F.col('Gasto_com_carnes') + F.col('Gasto_com_frutos_do_mar') + F.col('Gasto_com_doces') + F.col('Produtos_ouro'))

In [ ]:
#Mostrar a escolariade das pessoas e o total de compras
df_spark.groupBy('Escolaridade').sum('total_compras').show()

In [ ]:
#Mostrar o cliente mais velho
df_spark.orderBy(F.col('Ano_nascimento').asc()).show()

In [ ]:
#Mostrar o cliente mais jovem
df_spark.orderBy(F.col('Ano_nascimento').desc()).show()

In [ ]:
#Mostrar o cliente com a maior renda 
df_spark.orderBy(F.col('Renda').desc()).show()

In [ ]:
#Mostrar o cliente que gastou mais
df_spark.orderBy(F.col('total_gastos').desc()).show()

In [ ]:
#Verificar a qtde de pessoas e seu nivel de escolaridade
df_spark.groupBy('Escolaridade').count().show()

In [ ]:
#Mostrar a média da Renda familiar anual e o nivel de escolaridade
df_spark.groupBy('Escolaridade').agg(F.avg('Renda')).show()

***Finalizando o trabalho***

In [ ]:
#Transformando o dataframe Pyspark em csv
df_spark.toPandas().to_csv('gs://projetoindividual/Tratado', index=False)

In [ ]:
path_tratado = 'projetoindividual/Tratado/campanha_marketing.csv'

In [ ]:
df_tratado = pd.read_csv(path_tratado, sep=',')

In [ ]:
db = client['soulcode']

In [ ]:
collection = db['projeto_individual']

In [ ]:
df_dicio = df_tratado.to_dict("records")

In [ ]:
collection.insert_many(df_dicio)

In [ ]:
#enviando para o bucket GCP
df_dicio.to_csv('gs://projetoindividual/Tratado')